## Document Clustering with Python

The dataset is a list of the top 100 films of all time from an IMDB user list called Top 100 Greatest Movies of All Time (The Ultimate List) by [ChrisWalczyk55](https://www.imdb.com/list/ls055592025/). It had been originally scraped by Brandon Rose (see https://github.com/brandomr/document_cluster).

We will exploring different clustering methods to find out which movies (based on their synopsis) are grouped together.

In [ ]:
# importing libraries
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

### Setting up dataset

In [ ]:
#import lists: titles, imdb and wikipedia synopses
titles = open('../../DataDirectory/100movies/title_list.txt').read().split('\n')
#ensures that only the first 100 are read in
titles = titles[:100]

synopses_wiki = open('../../DataDirectory/100movies/synopses_list_wiki.txt').read().split('\n BREAKS HERE')
synopses_wiki = synopses_wiki[:100]

synopses_clean_wiki = []
for text in synopses_wiki:
    #strips html formatting and converts to unicode
    text = BeautifulSoup(text, 'html.parser').getText()
    synopses_clean_wiki.append(text)

synopses_wiki = synopses_clean_wiki

synopses_imdb = open('../../DataDirectory/100movies/synopses_list_imdb.txt').read().split('\n BREAKS HERE')
synopses_imdb = synopses_imdb[:100]

synopses_clean_imdb = []
for text in synopses_imdb:
    #strips html formatting and converts to unicode
    text = BeautifulSoup(text, 'html.parser').getText()
    synopses_clean_imdb.append(text)

synopses_imdb = synopses_clean_imdb

# combine the synopses 
synopses = []
for i in range(len(synopses_wiki)):
    item = synopses_wiki[i] + synopses_imdb[i]
    synopses.append(item)

#checks
print(str(len(titles)) + ' titles')
print(str(len(synopses_wiki)) + ' wiki synopses')
print(str(len(synopses_imdb)) + ' imdb synopses')
print(str(len(synopses)) + ' combined synopses')

#put everything into one data frame
data_array = np.array([titles, synopses])
data = pd.DataFrame(data=data_array).T
data.columns =['title', 'contents']
#we will use this for some inspections later
data['rank'] = data.index

### Constructing the term frequency features

RECAP: TF-IDF stands for "Term Frequency — Inverse Document Frequency"

The rationale behind TF-IDF is the following:

* a word that frequently appears in a document has more relevancy for that document, meaning that there is higher probability that the document is about or in relation to that specific word
* a word that frequently appears in more documents may prevent us from finding the right document in a collection; the word is relevant either for all documents or for none. Either way, it will not help us filter out a single document or a small subset of documents from the whole set.

So then TF-IDF is a score which is applied to every word in every document in our dataset. And for every word, the TF-IDF value increases with every appearance of the word in a document, but is gradually decreased with every appearance in other documents. And the maths for that is in the next section.

A couple things to note about the parameters used in the Sklearn TfidfVectorizer below:

* max_df: this is the maximum frequency within the documents a given feature can have to be used in the tfi-idf matrix. If the term is in greater than 80% of the documents it probably cares little meanining (in the context of film synopses)
* min_idf: this could be an integer (e.g. 5) and the term would have to be in at least 5 of the documents to be considered. Here we pass 0.2; the term must be in at least 20% of the document. It can be found that if we allowed a lower min_df we end up basing clustering on names--for example "Michael" or "Tom" are names found in several of the movies and the synopses use these names frequently, but the names carry no real meaning.
* ngram_range: this just means we'll look at unigrams, bigrams and trigrams 

In [ ]:
# Under the hood, TfidfVectorizer computes the word counts, IDF values, and Tf-idf scores all using the same dataset.
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    min_df = 0.2,
    max_df = 0.8,
    max_features = 200000,
    stop_words = 'english',
    ngram_range=(1,3)
)

text = tfidf.fit_transform(data.contents)
terms = tfidf.get_feature_names_out()

In [ ]:
import nltk
import re

#load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

### Demo Preprocessing

In [ ]:
tfidf_stemmed = TfidfVectorizer(
    min_df=0.2,
    max_df=0.8, 
    max_features=200000,
    stop_words='english',
    tokenizer=tokenize_and_stem, 
    ngram_range=(1,3)
)

text = tfidf_stemmed.fit_transform(data.contents)
terms = tfidf_stemmed.get_feature_names_out()

### k-Means clustering

Use Elbow method to find out number of clusters (SSE plot - Sum of Squared errors)

In [ ]:
# minibatch kmeans method - MiniBatchKMeans is faster, but gives slightly different results
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

def find_optimal_clusters(data, max_k):
    iters = range(1, max_k+1, 1)
    
    sse = []
    for k in iters:
        sse.append(MiniBatchKMeans(n_clusters=k, init_size=1024, batch_size=2048, random_state=20).fit(data).inertia_)
        #sse.append(KMeans(n_clusters=k, n_init=10, random_state=20).fit(data).inertia_)
        print('Fit {} clusters'.format(k))
        
    f, ax = plt.subplots(1, 1)
    ax.plot(iters, sse, marker='o')
    ax.set_xlabel('Cluster Centers')
    ax.set_xticks(iters)
    ax.set_xticklabels(iters)
    ax.set_ylabel('SSE')
    ax.set_title('SSE by Cluster Center Plot')
    
find_optimal_clusters(text, 10)

In [ ]:
from sklearn.cluster import KMeans

km_num_clusters = 5

#n_init Number of times the k-means algorithm is run with different centroid seeds
km = KMeans(n_clusters=km_num_clusters, n_init = 10, random_state=20)
km_clusters = km.fit_predict(text)

#### k-Means Results

Some preparations to investigate the clustering results.

a) How many films per cluster?

In [ ]:
data['km_cluster'] = km_clusters
data['km_cluster'].value_counts()

b) What is the average ranking per cluster? Original ranking (Top 100)

In [ ]:
grouped = data['rank'].groupby(data['km_cluster'])
grouped.mean()

c) List of terms and titles in each cluster? To investigate what was clustered together

In [ ]:
def get_top_keywords(features, clusters, labels, n_terms):
    df = pd.DataFrame(features.todense()).groupby(clusters).mean()
    
    for i,r in df.iterrows():
        print("\nCluster {} terms: ".format(i), end='')
        print(','.join([labels[t] for t in np.argsort(r)[-n_terms:]]), end='')
    
get_top_keywords(text, km_clusters, terms, 10)
print();
print();

km_titles = data.groupby('km_cluster').head(5).reset_index(drop=True)
for i in range(km_num_clusters):
    print("Cluster %d titles: " % i, end='')
    print(km_titles[km_titles['km_cluster'] == i]['title'].to_list())

### Hierarchical Clustering

#### With scipy: Cosine similarity and Ward

In [ ]:
#cluster hierarchy package from scipy
from scipy.cluster.hierarchy import ward, dendrogram
#need distance matrix
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(text)

#define the linkage_matrix using ward clustering pre-computed distances
ward_linkage_matrix = ward(dist) 

fig, ax = plt.subplots(figsize=(15, 20)) # set size
ax = dendrogram(ward_linkage_matrix, orientation="right", labels=titles);

plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')

plt.tight_layout() #show plot with tight layout

#uncomment below to save figure
plt.savefig('ward_clusters.png', dpi=200) #save figure as ward_clusters

In [ ]:
plt.close()

In [ ]:
# The fcluster method is used to predict labels on the data
from scipy.cluster.hierarchy import fcluster

ward_num_clusters = 5

ward_clusters = fcluster(
    ward_linkage_matrix, 
    ward_num_clusters, 
    criterion='maxclust'
)

# add WARD results
data['ward_cluster'] = ward_clusters

In [ ]:
get_top_keywords(text, ward_clusters, terms, 10)
print();
print();

ward_titles = data.groupby('ward_cluster').head(5).reset_index(drop=True)
for i in range(1, ward_num_clusters+1):
    print("Cluster %d titles: " % i, end='')
    print(ward_titles[ward_titles['ward_cluster'] == i]['title'].to_list())

#### Scipy Cosine similarity and average

In [ ]:
from scipy.cluster.hierarchy import average
#define the linkage_matrix using average clustering pre-computed distances
average_linkage_matrix = average(dist) 

import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(15, 20)) # set size
ax = dendrogram(average_linkage_matrix, orientation="right", labels=titles);

plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')

plt.tight_layout() #show plot with tight layout

#uncomment below to save figure
plt.savefig('average_clusters.png', dpi=200) #save figure as ward_clusters


#### With Sklearn

In [ ]:
from sklearn.cluster import AgglomerativeClustering

sklearn_num_clusters = 5

cluster_model = AgglomerativeClustering(
    n_clusters = sklearn_num_clusters, 
    metric = 'precomputed', 
    linkage = 'average')

sklearn_cluster = cluster_model.fit_predict(dist) #using cosine similarity distance matrix

# add sklearn results
data['sklearn_cluster'] = sklearn_cluster

In [ ]:
get_top_keywords(text, sklearn_cluster, terms, 10)
print();
print();

sklearn_titles = data.groupby('sklearn_cluster').head(5).reset_index(drop=True)
for i in range(sklearn_num_clusters):
    print("Cluster %d titles: " % i, end='')
    print(sklearn_titles[sklearn_titles['sklearn_cluster'] == i]['title'].to_list())